In [1]:
# Imports
import pandas as pd
import matplotlib.pyplot as plt
import seaborn as sns
%matplotlib inline

sns.set(style="whitegrid")

In [2]:
# Load data (adjust path if running from a different working directory)
path = '../data/raw/superstore.csv'
try:
    df = pd.read_csv(path, parse_dates=['Order Date'], low_memory=False)
except Exception:
    df = pd.read_csv(path, low_memory=False)
    df['Order Date'] = pd.to_datetime(df['Order Date'], errors='coerce')

# Coerce Sales to numeric and drop rows missing Order Date or Sales
df['Sales'] = pd.to_numeric(df['Sales'], errors='coerce')
df = df.dropna(subset=['Order Date', 'Sales'])
df.head()

,Row ID,Order ID,Order Date,Ship Date,Ship Mode,Customer ID,Customer Name,Segment,Country,City,State,Postal Code,Region,Product ID,Category,Sub-Category,Product Name,Sales
0,1,CA-2017-152156,08/11/2017,11/11/2017,Second Class,CG-12520,Claire Gute,Consumer,United States,Henderson,Kentucky,42420.0,South,FUR-BO-10001798,Furniture,Bookcases,Bush Somerset Collection Bookcase,261.9600
1,2,CA-2017-152156,08/11/2017,11/11/2017,Second Class,CG-12520,Claire Gute,Consumer,United States,Henderson,Kentucky,42420.0,South,FUR-CH-10000454,Furniture,Chairs,"Hon Deluxe Fabric Upholstered Stacking Chairs,...",731.9400
2,3,CA-2017-138688,12/06/2017,16/06/2017,Second Class,DV-13045,Darrin Van Huff,Corporate,United States,Los Angeles,California,90036.0,West,OFF-LA-10000240,Office Supplies,Labels,Self-Adhesive Address Labels for Typewriters b...,14.6200
3,4,US-2016-108966,11/10/2016,18/10/2016,Standard Class,SO-20335,Sean O'Donnell,Consumer,United States,Fort Lauderdale,Florida,33311.0,South,FUR-TA-10000577,Furniture,Tables,Bretford CR4500 Series Slim Rectangular Table,957.5775
4,5,US-2016-108966,11/10/2016,18/10/2016,Standard Class,SO-20335,Sean O'Donnell,Consumer,United States,Fort Lauderdale,Florida,33311.0,South,OFF-ST-10000760,Office Supplies,Storage,Eldon Fold 'N Roll Cart System,22.3680


## Aggregate to Monthly Level
Create a `Month` column (period-month) and sum `Sales` per month.

In [ ]:
# Monthly aggregation
df['Month'] = pd.to_datetime(df['Order Date'].dt.to_period('M').astype(str))
monthly = df.groupby('Month')['Sales'].sum().reset_index()
monthly = monthly.sort_values('Month')

In [ ]:
# Plot monthly total sales and a 3-month rolling average
plt.figure(figsize=(12,5))
plt.plot(monthly['Month'], monthly['Sales'], marker='o', label='Monthly Sales')
plt.plot(monthly['Month'], monthly['Sales'].rolling(window=3, center=False).mean(), label='3-month MA', linestyle='--')
plt.title('Monthly Total Sales')
plt.ylabel('Sales (USD)')
plt.xlabel('Month')
plt.legend()
plt.xticks(rotation=45)
plt.tight_layout()
plt.show()

## Month-over-Month Change
Compute percent change to highlight growth/decline between months.

In [ ]:
monthly['pct_change'] = monthly['Sales'].pct_change() * 100
monthly[['Month','Sales','pct_change']].head(12)

In [ ]:
# Plot percent change (bar)
plt.figure(figsize=(12,4))
sns.barplot(x='Month', y='pct_change', data=monthly, color='steelblue')
plt.xticks(rotation=45)
plt.ylabel('Percent Change (%)')
plt.title('Month-over-Month % Change in Sales')
plt.tight_layout()
plt.show()

## Seasonal / Category Breakdown (optional)
You can decompose by `Category` or `Segment` to see which groups drive monthly trends. The cell below shows how to compute monthly series per `Category`.

In [ ]:
# Monthly sales by Category (pivot table)
monthly_cat = df.groupby([pd.to_datetime(df['Order Date'].dt.to_period('M').astype(str)), 'Category'])['Sales'].sum().reset_index().pivot(index=0, columns=1, values='Sales').fillna(0)
monthly_cat.index.name = 'Month'
monthly_cat.head()

In [ ]:
# Plot monthly sales by Category (stacked area)
plt.figure(figsize=(12,5))
monthly_cat.plot.area(ax=plt.gca(), cmap='tab20')
plt.title('Monthly Sales by Category (stacked)')
plt.ylabel('Sales (USD)')
plt.xlabel('Month')
plt.legend(title='Category', bbox_to_anchor=(1.02, 1), loc='upper left')
plt.tight_layout()
plt.show()

## Interpretation & Next Steps
- The line plot and rolling average show overall trend and smoothing to reveal underlying direction.
- Percent-change bars highlight months with large increases or drops which merit investigation (promotions, seasonality, data issues).
- The stacked area by `Category` shows the contribution of each category to monthly totals.

Next possible analyses:
- Decompose time series into trend/seasonal/residual components (statsmodels)
- Drill into `State` or `Region` to find geographic drivers
- Compare year-over-year (seasonality) by aligning months across years
- Export `monthly` and `monthly_cat` to CSV in `data/processed/` for reporting.

In [ ]:
# Export processed CSVs to data/processed/
import os
out_dir = os.path.join('..','data','processed')
os.makedirs(out_dir, exist_ok=True)
monthly.to_csv(os.path.join(out_dir, 'monthly_sales.csv'), index=False)
print('Wrote', os.path.join(out_dir, 'monthly_sales.csv'))
if 'Segment' in df.columns:
    df.groupby('Segment')['Sales'].sum().reset_index().sort_values('Sales', ascending=False).to_csv(os.path.join(out_dir, 'sales_by_segment.csv'), index=False)
    print('Wrote', os.path.join(out_dir, 'sales_by_segment.csv'))
if 'Category' in df.columns:
    df.groupby('Category')['Sales'].sum().reset_index().sort_values('Sales', ascending=False).to_csv(os.path.join(out_dir, 'sales_by_category.csv'), index=False)
    print('Wrote', os.path.join(out_dir, 'sales_by_category.csv'))
    monthly_cat.to_csv(os.path.join(out_dir, 'monthly_sales_by_category.csv'))
    print('Wrote', os.path.join(out_dir, 'monthly_sales_by_category.csv'))
print('Export complete')